<a href="https://colab.research.google.com/github/kavyajeetbora/modern_geospatial_stack/blob/master/notebooks/DuckDB_in_Jupyter_Notebooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DuckDB in Jupyter Notebooks
A streamlined workflow for SQL analysis with DuckDB and Jupyter

## Library Import and Configuration

In [58]:
!pip install --quiet duckdb
!pip install --quiet jupysql
!pip install --quiet duckdb-engine
!pip install --quiet pandas
!pip install --quiet matplotlib
!pip install -q osmnx
!pip install -q pydeck

In [59]:
import duckdb
import pandas as pd
import geopandas as gpd
import shapely
import osmnx as ox
import pydeck as pdk
# No need to import sqlalchemy or duckdb_engine
#  JupySQL will use SQLAlchemy to auto-detect the driver needed based on your connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


We configure jupysql to return data as a Pandas dataframe and have less verbose output

In [60]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Connecting to DuckDB
Connect jupysql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

In [61]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [62]:
%%sql
INSTALL httpfs;
INSTALL spatial;

,Success


## Downloading Builings in small area

In [63]:
# W,S,E,N =  72.824548,19.19574,72.869386,19.231531

W,S,E,N = 72.8457167244,19.1174666804,72.8505208832,19.1231406236

public_transport = ox.features.features_from_bbox(bbox=(N,S,E,W), tags={'public_transport':'station'}).reset_index()
pb = public_transport[public_transport['element_type']=='node'].copy()
pb_gdf = pb[['public_transport', 'geometry']]
pb_gdf.sample(min(5,len(pb_gdf)))

,public_transport,geometry
0,station,POINT (72.84642 19.11970)
1,station,POINT (72.84879 19.12046)


In [67]:
pb_gdf.to_parquet('stations.parquet')

In [68]:
pb['element_type'].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


element_type
node    2
Name: count, dtype: int64

In [69]:
df = duckdb.read_parquet('''s3://overturemaps-us-west-2/release/2024-06-13-beta.0/theme=buildings/type=*/*''')
df.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['id',
 'geometry',
 'bbox',
 'version',
 'update_time',
 'sources',
 'subtype',
 'names',
 'class',
 'level',
 'has_parts',
 'height',
 'num_floors',
 'min_height',
 'min_floor',
 'facade_color',
 'facade_material',
 'roof_material',
 'roof_shape',
 'roof_direction',
 'roof_orientation',
 'roof_color',
 'roof_height']

In [ ]:
%%time

%%sql
LOAD spatial;
LOAD httpfs;


    SELECT
        *
    FROM read_parquet('s3://overturemaps-us-west-2/release/2024-06-13-beta.0/theme=buildings/type=*/*', filename=true, hive_partitioning=1) as buildings
    JOIN read_parquet('stations.parquet') as stations ON St_contains(ST_GeomFromWKB(buildings.geometry), ST_GeomFromWKB(stations.geometry));

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Intersection of builings and stations

[Spatial joins using geopandas](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html)

[Predicates of spatial joins in geopanda](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#binary-predicate-joins)

In [ ]:
buildings_gdf = gpd.read_file("buildings_mumbai.geojson").fillna("Na")
buildings_gdf.sjoin(pb_gdf, how='inner')

,id,level,height,geometry,index_right,public_transport
129,08b608b096140fff0200332fd99e2ab0,1.0,Na,"POLYGON ((72.84658 19.12116, 72.84633 19.11941...",0,station
214,08b608b09610cfff0200839516a02212,1.0,Na,"POLYGON ((72.84799 19.12082, 72.84819 19.12070...",1,station


## Spatial join using duckdb

In [ ]:
%%time

%%sql
LOAD spatial;
LOAD httpfs;

SELECT
    *
FROM read_parquet('buildings_mumbai.parquet') as buildings
JOIN read_parquet('stations.parquet') as stations ON stations.id = buildings.id;


# SELECT *, st_intersection(v.geom, b.geom) AS clipped
# FROM (SELECT * FROM read_parquet('stations.parquet')), v
# WHERE st_intersects(b.geom, v.geom);

RuntimeError: (duckdb.duckdb.BinderException) Binder Error: Table "stations" does not have a column named "id"
LINE 4: ...ROM read_parquet('buildings_mumbai.parquet') as buildings
JOIN read_parquet('stations.parquet') as stations ON stations.id = buildings.id;
                                                  ^
[SQL: SELECT
    *
FROM read_parquet('buildings_mumbai.parquet') as buildings
JOIN read_parquet('stations.parquet') as stations ON stations.id = buildings.id;]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
def create_map(W,S,E,N, geojson_file, point_layer):
    bbox_geom = shapely.geometry.box(W,S,E,N)
    boundary_json = eval(gpd.GeoSeries(bbox_geom).to_json())

    boundary_layer = pdk.Layer(
        "GeoJsonLayer",
        boundary_json,
        opacity=1,
        stroked=True,
        filled=False,
        get_line_color=[100, 0, 0]
    )

    layer = pdk.Layer(
        "GeoJsonLayer",
        geojson_file,
        opacity=1,
        stroked=True,
        filled=True,
        get_fill_color=[100, 200, 0],
        get_line_color=[0,100,0],
        pickable=True
    )


    ## Add point layers
    nodes = pdk.Layer(
        "GeoJsonLayer",
        point_layer,
        opacity=1,
        stroked=True,
        filled=True,
        get_fill_color=[200, 100, 156],
        get_line_color=[200, 100, 156],
        pickable=True
    )

    layers = [boundary_layer, layer, nodes]

    C = bbox_geom.centroid
    view_state = pdk.ViewState(latitude=C.y, longitude=C.x, zoom=13, bearing=0, pitch=45)
    # Render

    r = pdk.Deck(layers=layers, initial_view_state=view_state, tooltip = True)
    return r

In [ ]:
%%time
buildings_gdf = gpd.read_file("buildings_mumbai.geojson").fillna("Na")
print(buildings_gdf.shape[0])
#buildings_gdf = buildings_gdf.sample(1000)
geojson = eval(buildings_gdf.to_json())

nodes_geojson = eval(nodes_df.to_json())
pt_geojson = eval(pb_gdf.to_json())

379
CPU times: user 224 ms, sys: 5.77 ms, total: 229 ms
Wall time: 236 ms


In [ ]:
Map = create_map(W,S,E,N, geojson_file=geojson, point_layer=pt_geojson)
Map

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 19.120303652,
    "longitude": 72.84811880379999,
    "pitch": 45,
    "zoom": 13
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "data": {
        "bbox": [
          72.8457167244,
          19.1174666804,
          72.8505208832,
          19.1231406236
        ],
        "features": [
          {
            "bbox": [
              72.8457167244,
              19.1174666804,
              72.8505208832,
              19.1231406236
            ],
            "geometry": {
              "coordinates": [
                [
                  [
                    72.8505208832,
                    19.1174666804
                  ],
                  [
                    72.8505208832,
                    19.1231406236
                  ],
                  [
                    72.8457167244,
                    19.1231406236
                  ],
                  [
                    72.8457167244,
                    19.1174666804
                  ],
                  [
                    72.8505208832,
                    19.1174666804
                  ]
                ]
              ],
              "type": "Polygon"
            },
            "id": "0",
            "properties": {},
            "type": "Feature"
          }
        ],
        "type": "FeatureCollection"
      },
      "filled": false,
      "getLineColor": [
        100,
        0,
        0
      ],
      "id": "612e2d4e-ffd0-4156-8f24-98875cd54646",
      "opacity": 1,
      "stroked": true
    },
    {
      "@@type": "GeoJsonLayer",
      "data": {
        "features": [
          {
            "geometry": {
              "coordinates": [
                [
                  [
                    72.8462797090666,
                    19.1175705396462
                  ],
                  [
                    72.8462750419897,
                    19.1175315353275
                  ],
                  [
                    72.8462531521692,
                    19.1175339018413
                  ],
                  [
                    72.8462503550855,
                    19.1175105256497
                  ],
                  [
                    72.8464428206382,
                    19.1174897180568
                  ],
                  [
                    72.8464502848665,
                    19.1175520985664
                  ],
                  [
                    72.8462797090666,
                    19.1175705396462
                  ]
                ]
              ],
              "type": "Polygon"
            },
            "id": "0",
            "properties": {
              "height": "Na",
              "id": "08b608b096b91fff02007266a3fd1497",
              "level": "Na"
            },
            "type": "Feature"
          },
          {
            "geometry": {
              "coordinates": [
                [
                  [
                    72.8471163,
                    19.1176167
                  ],
                  [
                    72.847101,
                    19.1175309
                  ],
                  [
                    72.8474355,
                    19.1174777
                  ],
                  [
                    72.8474508,
                    19.1175635
                  ],
                  [
                    72.8471163,
                    19.1176167
                  ]
                ]
              ],
              "type": "Polygon"
            },
            "id": "1",
            "properties": {
              "height": "Na",
              "id": "08b608b0968c9fff02006ea3d95745f6",
              "level": "Na"
            },
            "type": "Feature"
          },
          {
            "geometry": {
              "coordinates": [
                [
                  [
                    72.8461388795148,
                    19.1176552751777
                  ],
                  [
           